## BRONZE TO SILVER LAYER


In [1]:
import pandas as pd
from global_paths import GlobalPaths
from common_utilities import replace_punctuation_from_columns

### TradeHistory Paths


In [2]:
# Initialize TradeHistory Paths
TradeHistoryPath = GlobalPaths(source_name="DATA", object_name="TradeHistory")
TradeHistoryBronzePath = TradeHistoryPath.createLayer(layer_name="BRONZE")
TradeHistorySilverPath = TradeHistoryPath.createLayer(layer_name="SILVER")
TradeHistorySilverFile = TradeHistorySilverPath.joinpath("TradeHistory_data.csv")

### Bronze Layer - Trade History


In [3]:

# Initialize an empty list to store DataFrames
dfs = []

# Loop through List of all CSV files in the folder
for file_path in TradeHistoryBronzePath.glob("*.csv"):
    # Read the CSV file
    df_bronze = pd.read_csv(file_path)
    # Append the DataFrame to the list
    dfs.append(df_bronze)

# Concatenate all DataFrames into one
df_TradeHistory = pd.concat(dfs, ignore_index=True)

df_TradeHistory.columns = replace_punctuation_from_columns(df_TradeHistory.columns)
df_TradeHistory.dropna(how="all", axis=1, inplace=True)

# Convert 'trade_num' to int
df_TradeHistory["trade_num"] = df_TradeHistory["trade_num"].fillna(0).astype(int)

# Remove the currency symbol and commas, then convert to float
df_TradeHistory["amount"] = (
    df_TradeHistory["amount"].replace("[₹,]", "", regex=True).astype(float)
)
df_TradeHistory["price"] = (
    df_TradeHistory["price"].replace("[₹,]", "", regex=True).astype(float)
)

# Add Datetime Col
df_TradeHistory["datetime"] = pd.to_datetime(
    df_TradeHistory["date"] + " " + df_TradeHistory["trade_time"].fillna("00:00:00"),
    format="%d-%m-%Y %H:%M:%S",
)

# Convert the 'expiry' column in df_TradeHistory from string to datetime format (day-month-year)
df_TradeHistory["expiry"] = pd.to_datetime(
    df_TradeHistory["expiry"], format="%d-%m-%Y"
)

# Convert the 'side' column in df_TradeHistory to uppercase
df_TradeHistory["side"] = df_TradeHistory["side"].str.upper()

# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by=["datetime", "company"])

df_TradeHistory = df_TradeHistory.rename(
    columns={
        "side": "trade_type",
        "quantity": "trade_quantity",
        "price": "trade_price",
    }
)


df_TradeHistory = df_TradeHistory[
    [
        "datetime",
        "trade_num",
        "exchange",
        "segment",
        "company",
        "instrument_type",
        "scrip_code",
        "strike_price",
        "expiry",
        "trade_type",
        "amount",
        "trade_quantity",
        "trade_price",
    ]
]

# Save the result as a csv file
df_TradeHistory.to_csv(TradeHistorySilverFile, index=None)
df_TradeHistory.info()

print("Silver Layer csv file for trade history successfully created at:")
print(TradeHistorySilverFile.resolve())

<class 'pandas.core.frame.DataFrame'>
Index: 139 entries, 17 to 42
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   datetime         139 non-null    datetime64[ns]
 1   trade_num        139 non-null    int32         
 2   exchange         139 non-null    object        
 3   segment          139 non-null    object        
 4   company          139 non-null    object        
 5   instrument_type  139 non-null    object        
 6   scrip_code       139 non-null    object        
 7   strike_price     139 non-null    object        
 8   expiry           93 non-null     datetime64[ns]
 9   trade_type       139 non-null    object        
 10  amount           139 non-null    float64       
 11  trade_quantity   139 non-null    float64       
 12  trade_price      139 non-null    float64       
dtypes: datetime64[ns](2), float64(3), int32(1), object(7)
memory usage: 14.7+ KB
Silver Layer csv file f